In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import time
import datetime as dt
from pathlib import Path
from data_functions import  create_causal_data


#### Adjust pandas Outcput

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#### LOAD DATA:
1. <b>CANARY(TREATED)
2. <b>BIOMETRICS(OUTCOME)
3. <b>PMPM(ELIGIBLE)

In [4]:
%%time
canary_p = '../../temp/CANARY.p'
biometrics_p = '../../data/BIOMETRICS'
pmpm_p = '../../temp/PMPM.p'

canary = pd.read_pickle(canary_p)
biometrics = pq.read_table(biometrics_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)


CPU times: user 27.3 s, sys: 9.64 s, total: 36.9 s
Wall time: 19.4 s


In [5]:
pmpm.head(2)

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  \
0  2570122               95      40392        0             0                  1       0          1                    0             0          0              1          0           0       1                 0            0                  0                  1                     0        1                   0              1                        0                  0            0                0         12                      0            0                0            1                  0                 1               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               1             0            1               0             0         0               0            0                   0                      0                      0           0                0              0               0   
1  2570122               94      19837        0             0                  0       0          1                    0             1          0              0          0           0       1                 0            0                  0                  0                     0        1                   0              0                        0                  0            0                0          7                      0            0                0            1                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               1             0            1               0             0         0               0            0                   0                      0                      0           0                0              0               0   

   CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR MBR_GNDR     MYR  
0            0                   0        0                 0                       1               0                0          0           0                   0                 0        M  202002  
1            0                   0        0                 0                       0               0                0          0           0                   0                 0        M  201810

#### An inspection of categories registered in biometrics data

In [6]:
%%time
pd.DataFrame(biometrics.groupby(by = 'BIOMETRIC_CATEGORY').apply(len)).T

CPU times: user 19.6 s, sys: 5.16 s, total: 24.8 s
Wall time: 24.8 s


BIOMETRIC_CATEGORY     A1C       BMI    BNP  Blood Pressure (Diastolic)  Blood Pressure (Systolic)  Cholesterol (LDL)  DCSI-EGFR  DCSI-MICROALBUMIN  DCSI-SERUM  Ferritin    Height  Hematocrit  Hemoglobin    Iron  NT_proBNP     PSA  Potassium  Troponin    Weight    eGFR
0                   790853  10034183  42552                     4902004                    5238991            1305880     628758              81297     2734084    219613  11808045     2574412     2919909  197878       9258  310283    2666981     94030  14000267  907656

## Steps in creating causal data
``` python
%%time
from data_functions import  biometrics_f, pmpm_f, canary_f, merge_filter_separate, control_outcomes, treated_outcomes, combine_treated_controls
biometrics = biometrics_f(biometrics, 'BMI', True)
pmpm = pmpm_f(pmpm, True, 12)
canary = canary_f(canary)
merged, treated, controls, canary = merge_filter_separate(biometrics, pmpm, canary, merge_how = 'inner')
controls_final = control_outcomes(controls, max_days_apart = 180, min_days_apart = 120)
treated_final, treated_help = treated_outcomes(treated, max_days_apart = 180, min_days_apart = 120)
df = combine_treated_controls(treated_final, control_finals)
```

##  A1C Category above 6.6 bad

In [7]:
%%time
th = 6.6
merge_how = 'inner'
control_filters = 'new'

df2, treated_help2 = create_causal_data(canary.copy(), pmpm.copy(), biometrics.copy(),
                      outcome_under_study = 'A1C', drop_single = True,
                      enroll_filt = True, enroll_months = 12, merge_how = merge_how,
                      max_days_apart = 180, min_days_apart = 120, threshold = th,
                                    bigger_better = False, window = 3,
                      control_filters = control_filters)

Starting Control Outcomes New 0.33s per 100 id processing
Total of: 60489 ids, time to finish: 181.46699999999998 s
CPU times: user 4min 20s, sys: 15.2 s, total: 4min 35s
Wall time: 4min 35s


## Create dataset for a Biometric Category (BMI) above 28 bad

In [8]:
%%time
th = 28
merge_how = 'inner'
control_filters = 'new'
df1, treated_help1 = create_causal_data(canary.copy(), pmpm.copy(), biometrics.copy(), 
                      outcome_under_study = 'BMI', drop_single = True,
                      enroll_filt = True, enroll_months = 12, merge_how = merge_how,
                      max_days_apart = 180, min_days_apart = 120, threshold = th,
                                       bigger_better = False, window = 3,
                                      control_filters = control_filters)

Starting Control Outcomes New 0.33s per 100 id processing
Total of: 161789 ids, time to finish: 485.367 s
CPU times: user 11min 3s, sys: 21.2 s, total: 11min 24s
Wall time: 11min 22s


In [9]:
Path('../../matching_data').mkdir(parents = True, exist_ok = True)
df1.to_pickle('../../matching_data/data.p')


In [10]:
df1.BIOMETRIC_CATEGORY.value_counts(), df2.BIOMETRIC_CATEGORY.value_counts()

(BMI    487335
 Name: BIOMETRIC_CATEGORY, dtype: int64, A1C    63918
 Name: BIOMETRIC_CATEGORY, dtype: int64)

In [11]:
df1.TREATED.value_counts(), df2.TREATED.value_counts()

(0    486962
 1       373
 Name: TREATED, dtype: int64, 0    63871
 1       47
 Name: TREATED, dtype: int64)

### <span style="color:red">Attention </span>
biometrics have datapoints from 1920, 1940... and one datapoint from the future 2179


### DoWhy demonstration on the data(install dowhy with)
```pip3 install dowhy --user```

In [12]:
import dowhy
from dowhy import CausalModel

import logging
logging.getLogger("dowhy").setLevel(logging.WARNING)
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

### Specify Common Causes Names

In [13]:
common_causes = np.setdiff1d(df1.columns, ['BIOMETRIC_CATEGORY', 'OUTCOME', 'OUTCOME2', 'OUTCOME3'
                                          'PERS_ID', 'TREATED', 'RESULT_VALUE2'])

df1.TREATED = df1.TREATED.astype(bool)
df2.TREATED = df2.TREATED.astype(bool)


### Create data dictionary for causal modeling

In [14]:
data1 = {'df':df1, 
        'common_causes_names':list(common_causes), 
        'outcome1':'OUTCOME', 
        'outcome2':'OUTCOME2',
        'outcome3':'OUTCOME3',
        'treatment_name':'TREATED'}

data2 = {'df':df2, 
        'common_causes_names':list(common_causes), 
        'outcome1':'OUTCOME', 
        'outcome2':'OUTCOME2',
        'outcome3':'OUTCOME3',
        'treatment_name':'TREATED'}

In [15]:
def causal_analysis(data, outcome_name,  method = "backdoor.propensity_score_matching"):
    
    #define causal model
    model = CausalModel(
            data=data['df'],
            treatment=data['treatment_name'],
            outcome=data[outcome_name],
            common_causes=data['common_causes_names'],
            )
    
    #check if it is possible the estimand to be estimated
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
    
    #calculate propensities
    propensity_match_estimate = model.estimate_effect(identified_estimand,
                            method_name = method,
                            test_significance = False,
                            confidence_intervals = False,
                            target_units="att")
    
    return propensity_match_estimate



In [16]:
m1 = causal_analysis(data1, 'outcome1')
m2 = causal_analysis(data2, 'outcome1')

### BMI RESULTS

In [17]:
m1.value

### A1C RESULTS

In [18]:
m2.value

In [19]:
df2[['TREATED', 'OUTCOME', 'RESULT_VALUE']][df2.TREATED].describe()

OUTCOME  RESULT_VALUE
count  47.000000     47.000000
mean    0.382979      5.501418
std     0.491369      0.960574
min     0.000000      1.500000
25%     0.000000      5.200000
50%     0.000000      5.400000
75%     1.000000      6.000000
max     1.000000      7.700000

In [20]:
df2[['TREATED', 'OUTCOME', 'RESULT_VALUE']][~df2.TREATED].describe()

OUTCOME  RESULT_VALUE
count  63871.000000  63871.000000
mean       0.483020      6.983471
std        0.499716      1.451155
min        0.000000      0.000000
25%        0.000000      6.000000
50%        0.000000      6.700000
75%        1.000000      7.600000
max        1.000000     17.000000

In [21]:
biometrics[biometrics.BIOMETRIC_CATEGORY == 'A1C'].RESULT_VALUE.astype('float').describe()

count    790853.000000
mean          6.544055
std           1.470449
min           0.000000
25%           5.600000
50%           6.200000
75%           7.100000
max          19.500000
Name: RESULT_VALUE, dtype: float64

In [22]:
df2

AGE_AT_MIDMONTH  ALLOW_AMT BIOMETRIC_CATEGORY  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0                   81        553                A1C        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          3                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0   
1                   76        677                A1C        0             0                  0       0          0                    0             0          0              0          0           0       0                 0            0                  0                  1                     0        0                   0              0                        0                  0            0                0          6                      0            0                0            1                  0                 1               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            0               0             0         0               0            0                   0                      0                      0           1                0              0   
2                   69        327                A1C        0             0                  0       0          0                    0             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            1                0          3                      0            0                0            1                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               0             0            0               0             0         0               0            0                   0                      0                      0           1                0              0   
3                   69        381  

In [23]:
df1[['TREATED', 'OUTCOME', 'RESULT_VALUE']][df1.TREATED].describe()

OUTCOME  RESULT_VALUE
count  373.000000    373.000000
mean     0.428954     33.013970
std      0.495592      5.880609
min      0.000000     17.180122
25%      0.000000     29.504742
50%      0.000000     31.951182
75%      1.000000     36.359291
max      1.000000     51.369088

In [24]:
df1[['TREATED', 'OUTCOME', 'RESULT_VALUE']][~df1.TREATED].describe()

OUTCOME   RESULT_VALUE
count  486962.000000  486962.000000
mean        0.489515      31.236902
std         0.499891       6.936777
min         0.000000      12.080000
25%         0.000000      26.430396
50%         0.000000      30.105430
75%         1.000000      35.034958
max         1.000000      59.990002